In [ ]:
import openpyxl
import pandas as pd
import re

In [ ]:
import os

In [ ]:
if not os.path.exists('output'):
    os.mkdir('output')

# Using OpenPyXL for Excel Workbooks

In this demo we'll showcase some of the features of [OpenPyXL](), a powerful Python library for reading and writing Excel files. OpenPyXL doesn't give us unlimited access to available Excel commands, but it does allow us to do things like format cells, create and modify sheets, and even generate charts. Moreover, it works even if a machine doesn't have Excel installed! 

This library can fit in a number of automated workflows including:
- Generating reports
- Checking data integrity
- Modifying structure of spreadsheets

We're going to focus on this last one.

<img src="img/example.png">

Take a look at the spreadsheet above. Let's say we want to take the contents of cell A10 and parse them into three pieces of information: period, campaign, and run date. Then, we want to create three new columns in the table at the bottom, and fill them with this information. 

For a single file, it would be trivial to do this by hand. But what if this is just one file among hundreds of the same template, and we've been asked to modify all of them? 

## Can't we just use Pandas?

Without even showing what would be involved in creating these columns in Pandas, let's just note what happens if we read in an Excel file and write it back to a new file without modifying anything.

In [ ]:
df = pd.read_excel('example.xlsx', header=None)
df

In [ ]:
df.to_excel('output/pandas-01.xlsx')

<img src="img/pandas-01.png">


### Get rid of column headers and index

In [ ]:
df.to_excel('output/pandas-02.xlsx', header=False, index=False)

<img src="img/pandas-02.png">


Pandas destroys all formatting when it reads in data from and Excel file. It does this with good reason - it is built for efficient computation, not for working specifically with Excel and designing spreadsheets.

But if we need to maintain formatting, we will need a different tool.

## Pandas vs OpenPyXL pros and cons

When should we use each library? 

- Pandas is much better suited for efficient data analysis but has limited functionality for writing Excel files. It is the gold standard for working with tabular data in Python. 

- OpenPyXL allows for sophisticated Excel operations but is VERY memory-intensive. It is better suited for automatically generating nicely-formatted reports or for automating the modification of many small Excel files.

Note also that the two can be used in conjunction with one another! 

## Opening a spreadsheet with OpenPyXL

In [ ]:
from openpyxl import load_workbook

We work with files in OpenPyXL in an object-oriented manner. At the most basic level, we have the following hierarchy of objects. There are other classes as well, e.g. Sheet could technically be a worksheet or a chart sheet, but this gives us a basic idea:

- Workbook
    - Sheet
        - Cell
        
First, we load a workbook. Then we define a variable `sheet` as the currently active sheet - whatever would be visible when opening the file.

In [ ]:
# Start by opening the spreadsheet and selecting the main sheet
workbook = load_workbook(filename="example.xlsx")
sheet = workbook.active


We can save a workbook with `workbook.save()`

In [ ]:
# Save the spreadsheet
workbook.save('output/openpyxl.xlsx')


<img src="img/openpyxl.png">
Note that it preserves the original formatting!

When we're done with a workbook, we close it with `workbook.close()` to free up memory. This is important, as OpenPyXL is very memory intensive! The documentation states that a 50MB notebook can use as much as 2.5GB of memory!

In [ ]:
workbook.close()


## Extracting data from specific cells

Let's get to work extracting the data from A10 and parsing it. 

In [ ]:
workbook = load_workbook(filename="example.xlsx")
sheet = workbook.active

We can extract a cell from a worksheet the same way we would perform a dictionary lookup in Python.

In [ ]:
cell = sheet['A10']
cell

Let's examine some of the contents of the cell.

In [ ]:
print(f'Coordinate: {cell.coordinate}')
print(f'Value: {cell.value}')

Everything looks normal so far. Let's assign the cell's value to a new variable `s`.

In [ ]:
s = sheet['A10'].value
s

### Clean the text

What happened? This doesn't look like it did above. The `\xa0` characters in the string represent sequences of **bytes**. More specifically, they represent a type of whitespace character in the **utf-8** Unicode string encoding.

We can remove these using Python's `unicodedata` module. 

In [ ]:
from unicodedata import normalize

In [ ]:
def clean_text(text):
    
    normalized = normalize('NFKC', text)
    return normalized.strip()


In [ ]:
text = clean_text(s)
text

### Extract the period, campaign, and run date

Here we use [Regular Expressions](https://en.wikipedia.org/wiki/Regular_expression) (regex) to parse the cell for the information we want. Regex is a rich subject with extremely powerful applications, but is outside the scope of this demo. 

For now, just note that we have the string `(.*)` three times in the expression below. These are telling regex which substrings we want to capture from the original string.

In [ ]:
def get_fields(text):
    
    regex = re.compile(r'Period: (.*) Campaign: (.*) Run Date: (.*)')
    return regex.match(text).groups()
    

In [ ]:
get_fields(text)

In [ ]:
period, campaign, run_date = get_fields(text)

print(period)
print(campaign)
print(run_date)

## Writing into a worksheet

In [ ]:
from openpyxl.utils import column_index_from_string

**Write a value to a single cell**

In [ ]:
sheet['I12'] = 'Period'
sheet['J12'] = 'Campaign'
sheet['K12'] = 'Run Date'

**Access cells by numerical coordinates**

In [ ]:
row = 12
col = column_index_from_string('I')
print(f'I translates to {col}')

sheet.cell(row, col).value

### What about writing to a range of cells?

If we want to write a value or values to multiple cells in OpenPyXL, the best we can do is to iterate over cell objects. 

This is most conveniently done with a numerical (column, row) notation, but we can do some extra work to translate between this and the typical notation for cell ranges we use in Excel.

Notably, 

In [ ]:
from openpyxl.utils import range_boundaries

In [ ]:
range_boundaries('A1:D3')

In [ ]:
def fill_range(sheet, range_string, value):
    
    boundaries = range_boundaries(range_string)
    col_start, row_start, col_end, row_end = boundaries
    
    
    for row in range(row_start, row_end+1):
        for column in range(col_start, col_end+1):
            sheet.cell(row, column).value = value
            

In [ ]:
fill_range(sheet, 'I13:I15', period)
fill_range(sheet, 'J13:J15', campaign)
fill_range(sheet, 'K13:K15', run_date)

In [ ]:
workbook.save('output/result-01.xlsx')

<img src="img/result-01.png">


## Formatting cells

OpenPyXL has rich support for formatting cells, including:

- Font
- Text alignment
- Number style
- Borders
- Fill color

and all of these can be modified individually. But we'll focus on a simple task here: copying the formatting of one cell to another.

In [ ]:
from copy import copy
from openpyxl.utils import coordinate_to_tuple

In [ ]:
def copy_style_to_range(sheet, source_coordinate, destination_range):
    
    c, r = openpyxl.utils.coordinate_to_tuple(source_coordinate)
    source = sheet.cell(c, r)
    boundaries = range_boundaries(destination_range)
    col_start, row_start, col_end, row_end = boundaries
    
    
    for row in range(row_start, row_end+1):
        for column in range(col_start, col_end+1):
            destination = sheet.cell(row, column)
            destination._style = copy(source._style)


In [ ]:
copy_style_to_range(sheet, 'H12', 'I12:K12')
copy_style_to_range(sheet, 'H13', 'I13:K15')
copy_style_to_range(sheet, 'H16', 'I16:K16')

In [ ]:
workbook.save('output/result-02.xlsx')
workbook.close()

<img src="img/result-02.png">
